# **Sentiment Analysis with LSTM**

This jupyter notebook performs sentiment analysis on the Rotten Tomatoes dataset using a **Long Short-Term Memory (LSTM)** neural network implemented in PyTorch. The dataset consists of phrases labeled with sentiment scores (0-4). Below is an overview of the workflow, model architecture, and results.

## **Workflow**

### **1. Import Libraries**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
#Check GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### **2. Prepare Data**
- Convert TF-IDF features to PyTorch tensors.
- Create a custom dataset class for handling the data.

In [ ]:
# Convert TF-IDF data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_vec2, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_val_vec2, dtype=torch.float32).to(device)

# Custom dataset class
class PhraseDataset(Dataset):
    def __init__(self, features, labels=None):
        self.features = features  # Features
        self.labels = labels  # Labels
        
    def __len__(self):
        return len(self.features)  # Dataset length
    
    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]  # Return features and labels
        else:
            return self.features[idx]  # Return only features

# Prepare dataset
train_labels = train['Sentiment'].values
train_dataset = PhraseDataset(X_train_tensor, train_labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32)

## **Model Architecture**

### **3. Define LSTM Model**

In [ ]:
class SentimentNN(nn.Module):
    def __init__(self, input_dim, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # First fully connected layer
        self.fc2 = nn.Linear(64, 32)  # Second fully connected layer
        self.fc3 = nn.Linear(32, output_size)  # Output layer
        self.dropout = nn.Dropout(0.5)  # Dropout layer to prevent overfitting

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = F.relu(self.fc2(x))  # Apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)  # Output layer
        return x

### **4. Initialize Model**

input_dim = X_train.shape[1]  # Input dimension
output_size = 5  # Number of output classes
net = SentimentNN(input_dim, output_size).to(device)  # Move model to device
net.train()  # Set model to training mode

### **5. Training**

In [ ]:
# Set Hyperparameters
epochs = 100  # Number of epochs
lr = 0.001  # Learning rate
optimizer = torch.optim.Adam(net.parameters(), lr=lr)  # Optimizer
criterion = nn.CrossEntropyLoss()  # Loss function

# Early Stopping
best_val_acc = 0  # Best validation accuracy
patience = 10  # Patience for early stopping
counter = 0  # Early stopping counter

# Training Loop
for e in range(epochs):
    net.train()  # Set model to training mode
    running_loss = 0.0  # Track training loss
    running_acc = 0.0  # Track training accuracy

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        optimizer.zero_grad()  # Zero gradients
        output = net(inputs)  # Forward pass
        loss = criterion(output, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()  # Accumulate loss
        running_acc += (output.argmax(dim=1) == labels).float().mean()  # Accumulate accuracy

    print(f"Epoch {e + 1}/{epochs}, Loss: {running_loss / len(train_loader):.6f}, Acc: {running_acc / len(train_loader):.6f}")

    # Validation phase
    net.eval()  # Set model to evaluation mode
    val_loss = 0.0  # Track validation loss
    val_acc = 0.0  # Track validation accuracy
    with torch.no_grad():  # Disable gradient calculation
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_output = net(val_inputs)  # Forward pass
            val_loss += criterion(val_output, val_labels).item()  # Accumulate loss
            val_acc += (val_output.argmax(dim=1) == val_labels).float().mean().item()  # Accumulate accuracy

    val_acc /= len(val_loader)  # Compute average validation accuracy
    print(f"Validation Loss: {val_loss / len(val_loader):.6f}, Validation Accuracy: {val_acc:.6f}")

    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc  # Update best validation accuracy
        counter = 0  # Reset early stopping counter
    else:
        counter += 1  # Increment early stopping counter
        if counter >= patience:
            print("Early stopping triggered.")  # Trigger early stopping
            break

### **6. Testing and Predictions**

In [ ]:
net.eval()  # Set model to evaluation mode
test_predictions = []  # Store test predictions
with torch.no_grad():  # Disable gradient calculation
    test_loader = DataLoader(PhraseDataset(X_test_tensor), batch_size=32)  # Create test data loader
    for test_inputs in test_loader:
        test_inputs = test_inputs.to(device)  # Move data to device
        test_output = net(test_inputs)  # Forward pass
        test_predictions.extend(test_output.argmax(dim=1).cpu().numpy())  # Store predictions

# Create output DataFrame
output_df = pd.DataFrame({
    'PhraseId': test['PhraseId'],  # Ensure 'PhraseId' matches the test set
    'Sentiment': test_predictions
})

# Save predictions to CSV
output_path = 'E:/shuju/answer/predictions.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create output directory if it doesn't exist
output_df.to_csv(output_path, index=False)  # Save predictions
print(f"Predictions saved to {output_path}")

## **Results**
- **Test Predictions**: Saved to `predictions.csv`.